### Ingestion de la carpeta "movie_company"

### Paso 1 - Leer los archivos CSV usando "DataFrameReader" de Spark

In [0]:
dbutils.widgets.text("p_enviroment", "")
v_enviroment = dbutils.widgets.get("p_enviroment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")


In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.functions import current_timestamp,lit,col
from pyspark.sql.types import StructType,StructField,IntegerType


In [0]:
movies_companies_schema = StructType([
    StructField("movieId", IntegerType(), True),
    StructField("companyId", IntegerType(), True)
    ])


In [0]:
movies_companies_df = spark.read.schema(movies_companies_schema).csv(f"{bronze_folder_path}/{v_file_date}/movie_company/")#Si tengo mas archivos en la carpeta y solo quiero cargar los del movie_company deberia usar movie_company_*.csv

In [0]:
movies_companies_df.printSchema()

In [0]:
display(movies_companies_df)

In [0]:
movies_companies_df.count()#Cuento los registros

#### Paso 2 - Renombrar las columnas y añadir nuevas columnas
1. "movieId" renombrar a "movie_id"
2. "companyId" renombrar a "company_id"
3. Agregar las columnas "ingestion_date" y "enviroment"

In [0]:
movies_companies_final_df = add_ingestion_date(movies_companies_df).withColumnsRenamed({"movieId":"movie_id","companyId":"company_id"}).withColumns({"enviroment":lit(v_enviroment),"file_date":lit(v_file_date)})

In [0]:
display(movies_companies_final_df)

#### Paso 4 - Escribir la salida en formato "Parquet"

In [0]:
merge_condition = 'tgt.movie_id = src.movie_id AND tgt.company_id = src.company_id AND tgt.file_date = src.file_date'
merge_delta_lake(movies_companies_final_df, 'movie_silver', 'movies_companies', silver_folder_path, merge_condition, 'file_date')

In [0]:
%fs
ls /mnt/historialpeliculas/silver/movies_companies/

In [0]:
%sql
SELECT file_date, COUNT(1)
FROM movie_silver.movies_companies
GROUP BY file_date;

In [0]:
dbutils.notebook.exit("Success")